## 初始化操作，加入必要引用
每次都需要第一个执行的

In [1]:
%matplotlib inline
from pymongo import MongoClient
from collections import defaultdict
import pandas as pd
import numpy as np
import jieba
import matplotlib
from matplotlib.widgets import Slider
import matplotlib.pyplot as plt
mongo_client = MongoClient()
db = mongo_client.jd
product_col = db['products']
comment_col = db['comments']
marked_col = db['markedComments']
user_col = db['users']
tags_col = db['tags']

## 将商品描述的词汇加入jieba的词典

加入jieba字典，并且计算各个词语的词频。

In [95]:
keys = defaultdict(lambda: 0)
for product in product_col.find().sort('_id', -1):
    for k in product.keys():
        keys[k] += 1
unused_words = ['commentCount', 'skuid', '材质/工艺', '_id', '功\u3000率（W）', '电\u3000压', 'scoreCount', 'productId']
for unused in unused_words:
    del keys[unused]
ready_to_add_words = list(keys.keys())
ready_to_add_words += ['材质', '工艺', '功率', '电压', '轻薄', 'win', 'Windows', 'Win', 'windows', '英雄联盟', '硬盘']

for w in ready_to_add_words:
    jieba.add_word(w, tag='nz')

- 根据词频，取出来前40个词
- 过滤一些没用的词
- 从数据库中的商品描述的词

In [99]:
import simplejson as json
top100 = {a[0]: a[1] for a in json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))[:200]}
del top100['总体']
del top100['帮别人']
del top100['京东']

product_keys = dict(keys)
product_keys['轻薄'] = 1
product_keys['触摸屏'] = 1
product_keys['win'] = 1
product_keys['windows'] = 1
product_keys['Windows'] = 1
product_keys['Win'] = 1
product_keys['英雄联盟'] = 1
product_keys['硬盘'] = 1

## 尝试直接判断

对已经标注过的评论数据提取关键词。

### 判断规则

1. 如果无法提取出关键词（名词）则认为是无效评论。
2. 关键词去数据库两个字典里找，如果出现，则加一。
3. 如果出现次数为0，则认为没有评价对象，则是无效评论。
3. 如果加了标签信息或者上传图片，则认为是有效评论。


In [96]:
import jieba.analyse
import simplejson as json
import math

results = {
    'pos': {'right': 0, 'wrong': 0},
    'neg': {'right': 0, 'wrong': 0}
}

for comment in marked_col.find():
    tags = jieba.analyse.extract_tags(comment['content'], topK=20, withWeight=True, allowPOS=('ns', 'n', 'nz'))
    predict = None
    
    if len(tags):
        maybe = 0
        for tag in tags:
            if tag[0] in top100 or tag[0] in product_keys:
                maybe += 1
        predict = 0 if maybe else 1
    else:
        predict = 1
    
    if comment.get('commentTags') or comment.get('imageCount') or comment.get('usefulVoteCount')>5 or len(comment.get('content')) > 80:
        predict = 0
    
    if comment['mark'] == 0:
        if predict == comment['mark']:
            results['pos']['right'] += 1
        else:
            results['pos']['wrong'] += 1
            #print('Marked: ', comment['mark'], 'Predict: ', predict, comment['content'], tags)
    else:
        if predict == comment['mark']:
            results['neg']['right'] += 1
        else:
            results['neg']['wrong'] += 1
            #print('Marked: ', comment['mark'], 'Predict: ', predict, comment['content'], tags)
    
print(results)

{'pos': {'right': 436, 'wrong': 15}, 'neg': {'right': 116, 'wrong': 44}}


## 生成特征函数

In [100]:
def generate_feature(comment, output=False):
    global pos
    tags = jieba.analyse.extract_tags(comment['content'], topK=20, withWeight=True, allowPOS=('ns', 'n', 'nz'))
    if output:
        print(tags)
    comment_feature = [
        len(comment.get('commentTags', [])),
        comment.get('imageCount', 0),
        comment.get('usefulVoteCount')
    ]
    
    for (i, l) in enumerate(comment_length):
        if len(comment.get('content')) > l:
            comment_feature.append(i)
            break
    else:
        comment_feature.append(3)
    maybe = 0
    w_fraq = {}
    for tag in tags:
        if tag[0] in top100 or tag[0] in product_keys:
            if tag[0] not in words_dict:
                words_dict[tag[0]] = pos
                pos += 1
            w_fraq[tag[0]] = tag[1]
            maybe += 1
    words_fraq.append(w_fraq)
    comment_feature.append(maybe)
    return comment_feature

## 使用Naive Bayes训练

## 使用SVM训练

In [97]:
from sklearn import svm

features = []
marked = []
contents = []
words_dict = {}
words_fraq = []
pos = 0
comment_length = [80, 40, 15, 0]

for c in marked_col.find():
    feature = generate_feature(c)
    features.append(feature)
    marked.append(c['mark'])
    contents.append(c['content'])

# for feature, words in zip(features, words_fraq):
#     new_features = [0] * len(words_dict)
#     for (word, weight) in words.items():
#         new_features[words_dict[word]] = weight
#     feature += new_features


clf = svm.SVC(probability=True, C=0.9, cache_size=400)
clf.fit(features, marked)

# print(len(top100), len(product_keys))

# print(clf.fit(features[:400], marked[:400]))
# count = 0
# for predict, mark, c in zip(clf.predict_proba(features[400:]), marked[400:], contents[400:]):
#     #print(predict)
#     ans = 1 if predict[1] > 0.6 else 0
#     if ans != mark:
#         count += 1
#         print('Mark: %d, Contents: %s' % (mark, c), predict)

# print('Wrong:', count)



SVC(C=0.9, cache_size=400, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [98]:
for c in comment_col.find()[:1000]:
    features = generate_feature(c)
    #print(features)
    r = clf.predict_proba([features])[0]
    
    if r[1] > 0.6:
        print(c['content'])

整天感觉不错
很不错
挺好能再便宜就好了。
很好！
6666666，就是裸机，啥都没有
虽然退货了，还是要给京车点赞，请问微软，高清显示什么时候能跟第三方兼容啊?
不错不错不错不错的啊
bucuogdfhdfjhdhd
帮公司采购的，同事用了说 不错 上班很好用
帮公司买的 同事说挺好用的 上班用很好
好评！好评！好评！好评！好评！好评！好评！
HP的，第二次购买了，还不错... 
还行
真心的卡得不行了
好好好好好好好好好
好！
儿子说还好。
还好
一晚上用下来，非常喜欢
快递很好，包装好，好好学习
还好设计合理，质量上乘！！
八天才到，说实话不忍心打低分
超超超级好用
不错不错不错不错
挺好的，
快，就是快！京东真棒！
不错
样子太妖了，凑合着用。
帮朋友买的，要是送电脑包就好了
不错不错、只是如果搭配8G的会更好一点！
确实是不错的变形本，值得购买
好
宝贝很好，快递小哥非常棒
...........
待以后问了人家怎么样再说
公司买的，还不错
妹妹说挺不错的
还可以
还不错，挺好的， 还不错，挺好的，
现在用着还不错，不知道将来怎样
用到现在觉得还行，还得看以后
非常好 准备再购两台
看起来很高大上！用起来也很流畅
很好，喜欢，京东一如既往的好！
硬盘空间只有48G, 跟号称的64G有差距，为什么？
不愧是移动的工作站，有分量，哈哈哈，小女子不要轻易尝试，女汉子嘛，我就不说了，呵呵?
图形工作站，够大，用起来很舒服。
货对版，很好，超值
卡的不要不要的，开个机用了2分钟
宝贝已收到超乎香的喜欢太爱它了
客服从来没有找到过
可以，满意。。。。。。。。
京东就是一个骗子，这么多年支持你，你们竟然这样！
物美价廉吧。总体来说还是不错的。
还行
不错，做工好
物有所值
太棒了，我很喜欢
没拆机呢，应该不错吧
帮朋友买的，不错
好评，卖家够意思，最后准时到了！帮大忙了。
好用好用好用好用，好沉好沉好沉好沉。
用着还行，等用时间长了看看怎么样
不错
，，还不错
有点重。其它 还不错。
帮同事买的，很惊喜，就是发热太快！
这个也是给弟弟买的，高版本的不太习惯，不过用的还挺好。
不错不错不错不错不错
还可以
很好 就是一点 我想要16年的新货 可惜是15年的。。
还好
买给妹妹的老妹很喜欢?
如一坊间谍战片774445599665
很好，下次还来！
好好好！值得拥有！
很好

## 标记重复数据
根据评论数据里面的用户注册时间进行聚合，然后通过编辑距离计算多个评论之间的相似程度。

In [20]:
from leven import levenshtein
from bson import ObjectId
pipelines = [
    {'$group': {'_id': "$userRegisterTime", 'count': {'$sum': 1}, 'contents': {'$push': {'id': '$_id', 'content': '$content'}}}},
    {'$match': {'count': {'$gt': 4}}}
]
count = 0
for user_comments in comment_col.aggregate(pipelines):
    last_comment = None
    for comment in user_comments['contents']:
        if len(comment['content']) > 7:
            if not last_comment:
                last_comment = comment
            else:
                distance = levenshtein(last_comment['content'], comment['content'])
                if distance < 5:
                    print(distance)
                    print(last_comment['content'], comment['content'])
                    comment_col.update_one({'_id': ObjectId(comment['id'])}, {'$set': {'duplicate': True}})
                    count += 1
            last_comment = comment

print('Total Count: ', count)

0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
性价比很高，很不错，买了60多台了。 性价比很高，很不错，买了60多台了。
0
很好不错有需要会再来的给公司买的 很好不错有需要会再来的给公司买的
0
东西不错，会继续关注 东西不错，会继续关注
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
东西不错，性价比高，性能稳定，外观漂亮！ 东西不错，性价比高，性能稳定，外观漂亮！
0
公司使用，产品还算不错，评价点个赞。 公司使用，产品还算不错，评价点个赞。
0
外观很漂亮 性价比高 外观很漂亮 性价比高
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。 喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。喜欢华硕的笔记本，稳定。
0
更多优惠及技术支持联系直线0411-82373589，刘相凤工程师协助订购！ 更多优惠及技术支持联系直线0411-82373589，刘相凤工程师协助订购！
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不错，非常满意。
0
产品不错，非常满意。 产品不

In [63]:
import jieba.analyse
import json
import math

pos = 0
words = {}
for (k, v) in json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))[:1000]:
    words[k] = (v, pos)
    pos += 1
fraq = []

def add_word(w):
    global pos
    if words.get(w) is not None:
        return words[w]
    return None

def convert_to_matrix():
    point_words = words.keys()
    results = []
    for single in fraq:
        single_result = [0]*(len(point_words) + 2)
        for (index, weight) in single.items():
            single_result[index] = weight
        results.append(single_result)
    return (list(point_words), results)


for comment in comment_col.find()[:50000]:
    single = {}
    for (tag, weight) in jieba.analyse.textrank(comment['content'], topK=20, withWeight=True, allowPOS=('nz')):
        index = add_word(tag)
        if index is not None:
            single[index[1]] = weight * math.log10(index[0])
    #single[-1] = comment['userLevelId']
    #single[-2] = comment['usefulVoteCount']
    fraq.append(single)

point_words, matrix = convert_to_matrix()
print(point_words[:10])
print('Words Count:', len(point_words))

['个性', '火影', '机都', '指纹', '次数', '频率', '市场', '联网', '寝室', '过瘾']
Words Count: 1000


## K-Means 进行聚类

In [103]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=10)
matrix = []
for comment in comment_col.find()[:50000]:
    feature = generate_feature(comment)
    matrix.append(feature)

s = clf.fit(matrix)
print(s)
print(clf.cluster_centers_)
print(clf.labels_)
#i = 1
# while i <= len(clf.labels_):
#     print (i, clf.labels_[i-1])
#     i = i + 1
print(clf.inertia_)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)
[[  1.45074648e-01   2.55860193e-01   1.99700014e-01   2.53550998e+00
    8.11601786e-01]
 [  0.00000000e+00   2.75000000e+00   1.98916667e+02   1.75000000e+00
    2.66666667e+00]
 [  0.00000000e+00   3.66666667e+00   6.33000000e+02   1.66666667e+00
    3.66666667e+00]
 [  3.29113924e-01   1.99367089e+00   3.69936709e+01   1.31012658e+00
    3.78481013e+00]
 [  1.20907114e+00   1.47840944e+00   7.84094439e-01   2.80832557e-01
    9.14383349e+00]
 [  2.22222222e-01   2.75555556e+00   9.09777778e+01   1.57777778e+00
    3.28888889e+00]
 [  0.00000000e+00   2.50000000e+00   3.88000000e+02   2.25000000e+00
    1.75000000e+00]
 [  1.76923077e-01   1.06153846e+00   4.48717949e-01   1.46534326e+00
    3.73937138e+00]
 [  3.16526611e-01   1.51260504e+00   1.07871148e+01   1.24929972e+00
    3.59243697e+00]
 [  3.90092392e+00   2.

In [104]:
results = defaultdict(lambda: [])
pos = 0
for comment in comment_col.find()[:40000]:
    results[clf.labels_[pos]].append(comment['content'])
    pos += 1

for (k, v) in results.items():
    print('Key:', k)
    print(v[:5])

Key: 0
['整天感觉不错', '嗯，，，，确实有点卡，可能是win8没用习惯吧！有点失望', '很不错', '买完就降价了，微信购买才3099多掏了两百块钱。本来想买宏基的显卡比较好一点，就是不带DVD所以放弃了。过段时间在发图了。', '挺好能再便宜就好了。']
Key: 1
['本本用两天，感觉还是很不错的', '电脑感觉被人试用过，可能没开机试用，系统都是开机才设置的，主要是键盘上有轻微痕迹，电源适配器上有很深的划痕，也是无语了，电脑本身非常棒，申请换个适配器', ' 挺不错的  任何一样都很流弊', '电脑非常的好，本来从网上买还有点担心，但是开机一看，硬盘采用了3个多小时，感觉非常的好，就是电池损耗到了四点几～但总体感觉非常好～', '分辨率高，屏幕清晰，外观沉稳，配置是真的，撸大湿测得分数十四万整，总体满意。升win10需要下载个组件才显示升级提示，然并卵，还要等几天或几周才推送。office获取失败，意思是还得装破解版office或者上wps了。快递大哥很负责，辛苦了！']
Key: 2
['外现大气（有图），启动win8系统一至二分钟，速度快，配置性价比高，配件齐。点赞！', '超赞的本本 一直喜欢华硕的东西 一直信赖京东的品质 特别是快递小哥 相当给力']
Key: 3
['比较薄，用起来非常好，性价比高', '美观，实用，运行快，物超所值，京东商城的真的很好！', '京东快递员态度极差，一直在容忍。签收前，连个包裹在最外面一层的快递塑料袋都不能打开。HP电脑本身不错。JD个别快递人员素质跟不上。那么好的网站购物体验，我想JD也投了很多钱，线下却遇到如此恶劣态度的快递人员。购物体验的快乐指数瞬间为0.', '一星给自己，电脑就那样，买来卡，开机我计时了，三分钟多点，打开鲁大师卡住了，白屏，下面有图。这款电脑没固态，我问客服这款啥固态回答我Msata下面有图，然后我买了个三星msata价格也不便宜，结过买来了，问sdd位置在哪（实在没找到这个插槽），让我找售后，算了客服智商感人，然后我自己找了下，找到了，发现是ngff接口，白白买了一个sdd我也是呵呵哒，下面有图', '绝对的好评，反应挺快的，屏幕大，看起来很舒适。朋友说性价比挺高，让人羡慕']
Key: 4
['笔记本很漂亮，运行速度也很好，尤其是做工，三星的东西还是有品质啊，很愉快的一次购

In [36]:
import json
r = {}
sum_fraq = defaultdict(lambda: 0)
for (skuid, contents) in results.items():
    r[skuid] = list(reversed(sorted(contents.items(), key=lambda x: x[1])))
    for (k,c) in contents.items():
        sum_fraq[k] += c
for (skuid, after) in r.items():
    product = product_col.find_one({'skuid': skuid})
    
json.dump(
    list(reversed(sorted(sum_fraq.items(), key=lambda x: x[1]))), 
    open('sorted_keys.json', mode='w', encoding='utf-8'),
    ensure_ascii=False,
    indent=2
)

In [2]:
import json
keys = json.load(open('sorted_keys.json', mode='r', encoding='utf-8'))
key_map = {n[0]: n[1] for n in keys}
key_map

{'军工': 3,
 '防脏': 1,
 '高质量': 14,
 '补水': 1,
 '独卡': 1,
 '份京豆': 1,
 '对方': 25,
 '平衡器': 1,
 '光线': 29,
 '手放': 1,
 '乞丐': 10,
 '代工': 13,
 '记笔记': 2,
 '下鲁': 4,
 '右手边': 12,
 '帮弟': 1,
 '死样': 1,
 '末治者': 1,
 '麻袋': 1,
 '代理': 3,
 '右手掌': 2,
 '水军': 3,
 '小卡': 1,
 '大学': 137,
 '新鲜': 14,
 '真版': 1,
 '货太': 1,
 '瑞都': 1,
 '半面': 1,
 '解决方案': 17,
 '通行证': 1,
 '茨品': 1,
 '位数': 2,
 '时能': 12,
 '线头': 2,
 '会顶': 1,
 '直流': 1,
 '题库': 1,
 '信心': 21,
 '外置': 43,
 '保价': 33,
 '弊端': 2,
 '件产品': 1,
 '热情': 237,
 '标题': 7,
 '货蒸价': 1,
 '精彩': 1,
 '女方': 1,
 '纸壳': 2,
 '业务知识': 1,
 '卡翻': 1,
 '妹夫': 1,
 '全部都是': 4,
 '专玩': 1,
 '待处理': 1,
 '水泡': 1,
 '乱提': 1,
 '产品包装': 4,
 '平民化': 1,
 '个空': 1,
 '额额': 6,
 '走路': 2,
 '红安': 2,
 '颈椎病': 5,
 '边缝': 1,
 '珠峰': 3,
 '预料': 15,
 '理解万岁': 2,
 '平分': 4,
 '妹用': 1,
 '老板': 248,
 '结实': 179,
 '小玩意儿': 1,
 '系统内存': 7,
 '杂声': 4,
 '升学': 1,
 '小黑点': 6,
 '中轻': 3,
 '临时工': 1,
 '角落': 6,
 '堂堂': 1,
 '满嘴': 1,
 '黄油': 1,
 '眼见': 1,
 '错款': 1,
 '先吧': 1,
 '公版': 5,
 '手气': 1,
 '温情': 1,
 '大根': 1,
 '出远门': 1,
 '系统日志': 2,
 '跳帧': 8,
 '热线电话': 2,
 '土鳖'